# Know Error
🟩 The bot tends to genrate multiple responese for a single reply (fixed)

🟩 Genrates multiple responses while working with media files (fixed)

🟩 Is able to remember images only time it is uploaded.(fixed)

🟩 Improving the video responses(Direct function added by google for video file)

🟩 Finding a way to keep the converstion for the videos conversation

🟨 If the user sends a gif through the inbuilt gif section, it's html page is downloaded

# Goal
1. Fix the 2000 char limit. ✅
2. Saving the chats. ✅
3. Only reply when tagged or replied ✅
4. Consinent chats ✅
5. Image input ✅
6. Mutimedia input ✅
7. Webhook

# d_bot
This notebook lets you run a discord bot powered by google gemini api, with conversation memory which includes audio, video, and images.

You can type **"!check_token"** to check how many tokens are being used.
<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1OQGPc2CsYpnBhNfKEl0O0cDT6uXAcQ8g?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  
</table>


In [ ]:
#@title <b>v-- If you play on mobile, tap this to open music player and play the white noise to keep tab running in the background. or your session might get disconnected
%%html
<b>Press play on the music player to keep the tab alive, then start block below (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [1]:
# @title Mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Install requirments (Restart)

In [ ]:
!pip install -q -U google-generativeai
!pip install Discord
!pip install python-magic
!pip install nest_asyncio
!pip install textract

Step 2: Get the api key

In [1]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [2]:
# @title Step 2.5: List available models
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

print("Now select any one of the model and paste it in the 'model_name' below")

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision
Now select any one of the model and paste it in the 'model_name' below


In [3]:
# Set the event listener for the dropdown change
# @title Model configuration
text_generation_config = {
    "temperature": 0.9,
    "top_p": 0.95,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

# Initial prompt
system_instruction = "." # @param {type:"string"}
model_name = "models/gemini-1.5-pro-latest" # @param {type:"string"}

# Create the model using the selected model name from the dropdown
model = genai.GenerativeModel(model_name = model_name, generation_config=text_generation_config, system_instruction=system_instruction, safety_settings=safety_settings)

In [4]:
# @title Functions

from random import choice, randint
import os
import requests
import textract
from urllib.parse import urlparse, unquote
import re
import cv2
import shutil
import mimetypes
import magic
import json
import pickle
from datetime import datetime, timedelta

def extract_text(file_path: str) -> str:
    """
    Extracts text from a document
    """
    # Extract text from the document
    text = textract.process(file_path).decode('utf-8')

    return text

def download_file(attachment_link: str, user_id: str) -> tuple:
    """
    Downloads the file, determines its type, and renames it with the correct extension.
    """
    try:
        # Parse the URL to get the file name without the extension
        parsed_link = urlparse(unquote(attachment_link))
        path = parsed_link.path
        original_filename = os.path.basename(path).split('?')[0]

        # Create a temporary filename with the user_id as prefix and no extension
        temp_filename = f'file_{user_id}'
        temp_filename_no_ext = temp_filename.rsplit('.', 1)[0]

        # Download the file
        response = requests.get(attachment_link)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Save the file without an extension
        with open(temp_filename_no_ext, 'wb') as f:
            f.write(response.content)

        print(f"File downloaded successfully: {temp_filename_no_ext}")

        # Determine the file type and the correct extension
        mime_type = determine_file_type(temp_filename_no_ext)
        extension = mimetypes.guess_extension(mime_type) or '.bin'  # Default to .bin if unknown

        # Rename the file with the correct extension
        final_filename = f'{temp_filename_no_ext}{extension}'
        os.rename(temp_filename_no_ext, final_filename)
        print(f"File renamed to: {final_filename} with MIME type: {mime_type}")

        return extension, final_filename
    except requests.RequestException as e:
        print(f"Failed to download file. Error: {e}")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

def determine_file_type(filepath: str) -> str:
    """
    Determines the MIME type of a file by reading its contents.
    """
    try:
        # Initialize the magic library
        mime = magic.Magic(mime=True)
        mime_type = mime.from_file(filepath)
        return mime_type
    except Exception as e:
        print(f"Could not determine file type. Error: {e}")
        return 'application/octet-stream'


# Function to load chat history from a file
def load_chat_history(user_id, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'

    if not os.path.exists(full_path):
        with open(full_path, 'wb') as file:
            pickle.dump([], file)

    # Open the file and load the chat history
    with open(full_path, 'rb') as file:
        chat_history = pickle.load(file)

    return chat_history


# Function to save the chat history from a file
def save_chat_history(user_id, chat, custom_file_path):
    full_path = f'{custom_file_path}{user_id}_chat_history.pkl'
    with open(full_path, 'wb') as file:
        pickle.dump(chat.history, file)

def save_filetwo(user_id, time_file_path, url):
    file_path = f'{time_file_path}{user_id}_files_metadata.json'

    # Ensure the file exists and contains a valid JSON list; if not, initialize it
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        with open(file_path, 'w') as file:
            json.dump([], file)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []

    # Append the new data
    new_data = {
        'file_uri': url,
        'timestamp': datetime.now().isoformat()
    }
    data.append(new_data)
    print(file_path)

    # Write the updated list back to the file
    with open(file_path, 'w') as file:
        json.dump(data, file, indent=4)
        print("Successful saved the file url and upload time")

def check_expired_files(user_id, time_file_path):
    #check if the url is expired
    file_path = f'{time_file_path}{user_id}_files_metadata.json'
    if not os.path.exists(file_path) or os.stat(file_path).st_size == 0:
        with open(file_path, 'w') as file:
            json.dump([], file)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
        except json.JSONDecodeError:
            data = []
    current_time = datetime.utcnow()
    expired_files = []

    for entry in data:
        upload_time = datetime.fromisoformat(entry['timestamp'])
        if current_time - upload_time > timedelta(hours=48):
            expired_files.append(entry)

    return expired_files


# Running The Bot

In [ ]:
import re
import aiohttp
from typing import Final
import os
import discord
from discord import Intents, Client, Message
from google.colab import userdata
import PIL.Image
import nest_asyncio

# STEP 0: LOAD OUR TOKEN FROM SOMEWHERE SAFE
TOKEN: Final[str] = userdata.get('DISCORD_TOKEN')

# STEP 1: BOT SETUP
intents: Intents = Intents.default()
intents.message_content = True
client: Client = Client(intents=intents)

# STEP 2: MESSAGE FUNCTIONALITY
async def send_message(message: Message, response: str, user_id: str) -> None:
    # Check if the entire response is within the Discord character limit
    if len(response) <= 2000:
        await message.channel.send(response)
    else:
        # Initialize the start index of the chunk
        start_index = 0
        while start_index < len(response):
            # Find the end index by looking for the last space before the 2000 character limit
            end_index = response.rfind(' ', start_index, start_index + 2000)
            # If no space is found, just cut at the 2000 character limit
            if end_index == -1:
                end_index = start_index + 2000
            # Extract the chunk of text
            chunk = response[start_index:end_index]
            # Send the chunk as a separate message to the channel
            await message.channel.send(chunk)
            # Update the start index to the end of the last chunk
            start_index = end_index + 1  # Skip the space

# STEP 3: HANDLING THE STARTUP FOR OUR BOT
@client.event
async def on_message(message: Message) -> None:
    # Ignore messages sent by the bot itself
    if message.author == client.user:
        return

    # Check if the message is in a DM channel
    is_dm = isinstance(message.channel, discord.DMChannel)

    # Check if the bot is mentioned or if the message is a reply to the bot
    bot_mentioned = client.user in message.mentions
    is_reply = message.reference is not None and message.reference.resolved.author == client.user

    # Regular expression to detect URLs
    url_pattern = re.compile(r'(https?://[^\s]+)')
    urls = url_pattern.findall(message.content)
    Direct_upload = False
    Link_upload = False
    attach_url = None
    custom_path = '/content/drive/MyDrive/Discord_bot/'
    time_file_path = '/content/drive/MyDrive/Discord_bot/Time_files/'
    tempoery = []

    # Handle !check_token command
    if message.content.strip() == "!check_token":
        user_id: str = str(message.author.id)  # Get the user's ID as a string
        chat_history = load_chat_history(user_id, custom_path)
        no_files = check_expired_files(user_id, time_file_path)
        #extracting the expried urls
        for dct in no_files:
            tempoery.append(dct['file_uri'])
        for link in tempoery:
            target_word = (f'{link}')
            chat_history = [entry for entry in chat_history if target_word not in str(entry)]
            print(f'Succesfully removed: {target_word}')
            file_path = f'{time_file_path}{user_id}_files_metadata.json'
            with open(file_path, 'r') as file:

                data = json.load(file)
                for entry in data:
                    data = [entry for entry in data if target_word not in str(entry)]

            with open(file_path, 'w') as file:
                #updateing the url file
                json.dump(data, file, indent=4)
                print("Successful removed the file_uri")
        chat = model.start_chat(history=chat_history)
        #token_count = model.count_tokens(chat.history)
        response = f"{model.count_tokens(chat.history)}"
        await send_message(message, response, user_id)
        return

    # Respond only if the bot is mentioned, if it's a reply to the bot's message, if a URL is detected,
    # or if the message is in a DM channel
    if bot_mentioned or is_reply or is_dm:
        user_id: str = str(message.author.id)  # Get the user's ID as a string
        username: str = str(message.author)
        channel: str = str(message.channel)

        # Check for attachments in the message
        if urls:
            attach_url = urls[0]
            Link_upload = True

        if message.attachments:
            for attachment in message.attachments:
                attach_url = attachment.url
                Direct_upload = True
                break


        #Loading the chat history
        print(f"({user_id}): {message.content}")
        chat_history = load_chat_history(user_id, custom_path)
        no_files = check_expired_files(user_id, time_file_path)
        for dct in no_files:
              tempoery.append(dct['file_uri'])
        for link in tempoery:
            target_word = (f'{link}')
            chat_history = [entry for entry in chat_history if target_word not in str(entry)]
            print(f'Succesfully removed: {target_word}')
            file_path = f'{time_file_path}{user_id}_files_metadata.json'
            with open(file_path, 'r') as file:

                data = json.load(file)
                for entry in data:
                    data = [entry for entry in data if target_word not in str(entry)]

            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
                print("Successful removed the file_uri")


        chat = model.start_chat(history=chat_history)

        if Direct_upload or Link_upload:
            format, downloaded_file = download_file(attach_url, user_id)


            # Downloading/extracting text from the file while returning the file type
            if format in ('.pdf', '.docx', '.txt'):
                text = extract_text(downloaded_file)
                message.content = f'The user has uploaded a document: {text} The document has ended!! The current user input is: {message.content}'

                response = chat.send_message(message.content)
                response = response.text

                save_chat_history(user_id, chat, custom_path)
                await send_message(message, response, user_id)
                print(f"Bot: {response}")

            else:
                media_file = genai.upload_file(path=downloaded_file)
                save_filetwo(user_id, time_file_path, media_file.uri)

                response = chat.send_message([message.content, media_file])
                response = response.text

                save_chat_history(user_id, chat, custom_path)
                await send_message(message, response, user_id)
                print(f"Bot: {response}")
            Direct_upload = False
            Link_upload = False

        else:
            response = chat.send_message(message.content)
            response = response.text

            save_chat_history(user_id, chat, custom_path)
            await send_message(message, response, user_id)
            print(f"Bot: {response}")

# STEP 5: MAIN ENTRY POINT
async def main() -> None:
    await client.start(TOKEN)

# To run in a cloud service like Google Colab, Kaggle, etc.
nest_asyncio.apply()

# Run the main function
await main()

(For debugging)

In [ ]:
fullpath = '/content/drive/MyDrive/Discord_bot/738618866699075595_chat_history.pkl'

In [ ]:
with open(fullpath, 'rb') as file:
    chat_history = pickle.load(file)

In [ ]:
print(chat_history)

In [ ]:
print(chat_history)